# load packages

In [1]:
import os
import matplotlib.pyplot as plt
from skimage import io
import numpy as np
from skimage.morphology import disk, remove_small_objects, binary_opening
from scipy import ndimage as ndi
from skimage.measure import regionprops
import pandas as pd
import matplotlib.patches as patches
import pickle

# define functions

In [2]:
class class_seed:
    def __init__(self):
        self.seed_flag = 0  # check whether the seed is in imbibition stage or germination stage
        self.seed_coords = ''
        self.yolo_box = ''
        self.yolo_conf = ''
        self.yolo_label = ''

    def update_seed_coords(self, seedcoat_coords):
        self.seed_coords = seedcoat_coords
    
    def update_yolo(self, yolo_box, yolo_conf, yolo_label):
        self.yolo_box = yolo_box
        self.yolo_conf = yolo_conf
        self.yolo_label = yolo_label

# initialization

In [3]:
color_dict = {
    0:'red', # IMB (imbibition)
    1:'blue', # PRO (protrusion)
    2:'green', # RE (radicle emergence)
    3:'orange', # SE (seedling establishment)
}

In [4]:
input_dir = './example_G7/'
out_dir = './traits/phase'
yolo_adjust_dir = './traits/roi_adjust'
mask_overall_dir = './model_predict/G7_overall/'
mask_seed_dir = './model_predict/G7_seed/'
yolo_pickle_dir = './model_predict/G7_phase/'
row_num = 5
col_num = 5

In [5]:
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
if not os.path.exists(yolo_adjust_dir):
    os.makedirs(yolo_adjust_dir)

In [6]:
files = sorted(os.listdir(input_dir))
seed_class_list = []
previous_mask = ''
seed_num = 0
box_posess = []

In [7]:
for file_index, file_name in enumerate(files):
    print('*' * 40 + file_name + '*' * 40)
    file_prefix = file_name.split('.')[0]
    tmp_f = open(os.path.join(yolo_pickle_dir, f'{file_prefix}.pkl'), 'rb')
    tmp_yolo_box = pickle.load(tmp_f)
    tmp_yolo_box = [box for box in tmp_yolo_box if box.conf >= 0.5]
    box_possess = [0 for box in tmp_yolo_box]
    seed_mask = io.imread(os.path.join(mask_seed_dir, file_prefix+'.png')) > 0
    seed_mask = binary_opening(seed_mask, disk(2))
    image_origion = io.imread(os.path.join(input_dir, file_name))
    if file_index == 0:  # 1st image of the time series data
        label_seed, label_num = ndi.label(seed_mask)
        seed_num = label_num
        seed_class_list = [class_seed() for _ in range(seed_num)]
        centroid_rr, centroid_cc, seedregion_list = [], [], []
        for tmp_region in regionprops(label_seed):
            rr, cc = tmp_region.centroid
            centroid_rr.append(int(rr))
            centroid_cc.append(int(cc))
            seedregion_list.append(tmp_region)
        
        # sort & give each seed an ID
        data = {'cen_r': centroid_rr, 'cen_c': centroid_cc, 'seed_props': seedregion_list}
        cen_df = pd.DataFrame(data)
        df1 = cen_df[0:col_num].sort_values(by=['cen_c'])
        for i in range(1, row_num):
            temp = cen_df[i * col_num:(i + 1) * col_num].sort_values(by=['cen_c'])
            df1 = pd.concat([df1, temp], axis=0)
        df1 = df1.reset_index(drop=True)
        
        for i in range(label_num):
            seed_temp = df1.loc[i, 'seed_props']
            seed_class_list[i].update_seed_coords(seed_temp.coords)

            box_candidate_index = []
            box_overlap_area = []
            box_coords = []
            box_conf = []
            box_label = []
            for box_index, box in enumerate(tmp_yolo_box):
                box_coord = box.xyxy.tolist()[0]
                left, top, right, bottom = [int(m) for m in box_coord]
                tmp_box_mask = np.zeros(seed_mask.shape)
                tmp_box_mask[top:bottom+1, left:right+1]=1
                single_seed_mask_full = np.zeros(seed_mask.shape)
                single_seed_mask_full[seed_temp.coords[:,0], seed_temp.coords[:,1]] = 1
                new_overlap_mask = tmp_box_mask * single_seed_mask_full
                overlap_pixel_num = np.sum(new_overlap_mask)
                if overlap_pixel_num > 0:
                    box_candidate_index.append(box_index)
                    box_overlap_area.append(overlap_pixel_num)
                    box_conf.append(box.conf.tolist()[0])
                    box_label.append(box.cls.tolist()[0])
                    box_coords.append([left, top, right, bottom])
            if len(box_overlap_area) > 0:
                box_conf_sort = sorted(range(len(box_conf)), key=lambda k: box_conf[k], reverse=True)
                chosen_index = box_conf_sort[0]
                seed_class_list[i].update_yolo(box_coords[chosen_index], box_conf[chosen_index], box_label[chosen_index])

        previous_mask = seed_mask
        
        fig, ax = plt.subplots()
        plt.axis('off')
        ax.imshow(image_origion)
        for tmp_i in range(len(seed_class_list)):
            left, top, right, bottom = seed_class_list[tmp_i].yolo_box
            label_tmpbox = seed_class_list[tmp_i].yolo_label
            conf_tmpbox = seed_class_list[tmp_i].yolo_conf
            tmp_color = color_dict[label_tmpbox]
            width = right - left
            height = bottom - top
            ax.add_patch(
                patches.Rectangle((left, top), width, height, edgecolor=tmp_color, fill=False)
            )
            plt.text(left+5, top-10, 
                round(conf_tmpbox,2), 
                color='white', 
                fontsize=5, 
                fontweight='bold', 
                ha='left', 
                bbox=dict(facecolor=tmp_color, edgecolor='none', pad=1.5))

        plt.savefig(os.path.join(out_dir, file_name), dpi=300, bbox_inches='tight', pad_inches=0)
        plt.close() 
        
        with open(os.path.join(yolo_adjust_dir, f'{file_prefix}.pkl'), 'wb') as f_pickle:
            pickle.dump(seed_class_list, f_pickle)
        
        continue

    overall_mask = io.imread(os.path.join(mask_overall_dir, file_prefix+'.png')) > 0

    sub = np.logical_and(overall_mask, np.logical_not(seed_mask))
    sub2 = remove_small_objects(sub, 50)
    protrusion_flag = 1 if np.any(sub2 == 1) else 0

    if protrusion_flag == 0:  # imbibed seeds
        # relocalize seed
        mask_and = previous_mask * seed_mask
        tmp_overlap_img, tmp_overlap_num = ndi.label(mask_and)
        overlap_region_list = [region.coords[0] for region in regionprops(tmp_overlap_img)]

        seed_new_list = [_ for _ in regionprops(ndi.label(seed_mask)[0])]

        trans_dict = {}
        for i in range(seed_num):
            for rr, cc in overlap_region_list:
                if (rr, cc) not in trans_dict and \
                        np.any((seed_class_list[i].seed_coords == [rr, cc]).all(axis=1) == 1):
                    trans_dict[rr, cc] = i
                    break

        trans_new = {}
        for region_index, region_props in enumerate(seed_new_list):
            origin_seed_num = []
            for (rr, cc) in trans_dict.keys():
                if np.any((region_props.coords == [rr, cc]).all(axis=1) == 1):
                    origin_seed_num.append(trans_dict[rr, cc])
            origin_seed_num = list(set(origin_seed_num))
            if len(origin_seed_num) == 1:  # 唯一匹配
                trans_new[region_index] = origin_seed_num[0]

        for tmp_region_index, tmp_seed_same in trans_new.items():
            seed_class_list[tmp_seed_same].update_seed_coords(seed_new_list[tmp_region_index].coords)

            box_candidate_index = []
            box_overlap_area = []
            box_coords = []
            box_conf = []
            box_label = []
            for box_index, box in enumerate(tmp_yolo_box):
                if box_possess[box_index] > 0:
                    continue
                box_coord = box.xyxy.tolist()[0]
                left, top, right, bottom = [int(m) for m in box_coord]
                tmp_box_mask = np.zeros(seed_mask.shape)
                tmp_box_mask[top:bottom+1, left:right+1]=1
                single_seed_mask_full = np.zeros(seed_mask.shape)
                seed_coords = seed_class_list[tmp_seed_same].seed_coords
                single_seed_mask_full[seed_coords[:,0], seed_coords[:,1]] = 1
                new_overlap_mask = tmp_box_mask * single_seed_mask_full
                overlap_pixel_num = np.sum(new_overlap_mask)
                if overlap_pixel_num >= 0.85 * len(seed_coords):
                    box_candidate_index.append(box_index)
                    box_overlap_area.append(np.sum(tmp_box_mask * seedling_mask))
                    box_coords.append([left, top, right, bottom])
                    box_conf.append(box.conf.tolist()[0])
                    box_label.append(box.cls.tolist()[0])
            if len(box_overlap_area) > 0:
                box_conf_sort = sorted(range(len(box_conf)), key=lambda k: box_conf[k], reverse=True)
                chosen_index = box_conf_sort[0]
                seed_class_list[tmp_seed_same].update_yolo(box_coords[chosen_index], box_conf[chosen_index], box_label[chosen_index])
                box_possess[box_candidate_index[chosen_index]] = 1
    else:
        seed_new_list, seed_centroid_list = [], []
        for _ in regionprops(ndi.label(seed_mask)[0]):
            overlap_mask = np.zeros(overall_mask.shape)
            overlap_mask[_.coords[:, 0], _.coords[:, 1]] = 1
            overlap_mask = np.logical_and(previous_mask * overlap_mask, overall_mask)
            overlap_mask = remove_small_objects(overlap_mask, 50)
            if len(np.argwhere(overlap_mask > 0)) == 0: 
                continue
            seed_new_list.append(_)
            seed_centroid_list.append(np.argwhere(overlap_mask > 0)[0])

        trans_dict = {}
        for i in range(seed_num):
            for [rr, cc] in seed_centroid_list:
                if (rr, cc) not in trans_dict and \
                        np.any((seed_class_list[i].seed_coords == [rr, cc]).all(axis=1) == 1):
                    trans_dict[(rr, cc)] = i

        trans_new = {}
        for region_index, region_props in enumerate(seed_new_list):
            origin_seed_num = []
            for (rr, cc) in trans_dict.keys():
                if np.any((region_props.coords == [rr, cc]).all(axis=1) == 1):
                    origin_seed_num.append(trans_dict[(rr, cc)])
            origin_seed_num = list(set(origin_seed_num))
            if len(origin_seed_num) == 1:
                trans_new[region_index] = origin_seed_num[0]

        for tmp_region in regionprops(ndi.label(overall_mask)[0]):
            overlap_num = []
            tmp_coords = tmp_region.coords
            for point_index, point in enumerate(seed_centroid_list):
                rr, cc = point
                if np.any((tmp_coords == [rr, cc]).all(axis=1) == 1):
                    overlap_num.append(trans_new[point_index])
            if len(overlap_num) == 1:  # no overlap conditions
                tmp_region_index, tmp_seed_same = '', ''
                for temp_region_index, temp_seed_same in trans_new.items():
                    if temp_seed_same == overlap_num[0]:
                        tmp_region_index = temp_region_index
                        tmp_seed_same = overlap_num[0]
                        break
                seed_class_list[tmp_seed_same].update_seed_coords(seed_new_list[tmp_region_index].coords)
                temp_single_seed_mask = np.zeros(seed_mask.shape)
                temp_single_seed_mask[seed_new_list[tmp_region_index].coords[:, 0],
                seed_new_list[tmp_region_index].coords[:, 1]] = 1

                box_candidate_index = []
                box_overlap_area = []
                box_coords = []
                box_conf = []
                box_label = []
                for box_index, box in enumerate(tmp_yolo_box):
                    if box_possess[box_index] > 0:
                        continue
                    box_coord = box.xyxy.tolist()[0]
                    left, top, right, bottom = [int(m) for m in box_coord]
                    tmp_box_mask = np.zeros(seed_mask.shape)
                    tmp_box_mask[top:bottom+1, left:right+1]=1
                    new_overlap_mask = tmp_box_mask * temp_single_seed_mask
                    overlap_pixel_num = np.sum(new_overlap_mask)
                    if overlap_pixel_num >= 0.85 * np.sum(temp_single_seed_mask):
                        a, b = ndi.label(tmp_box_mask * seed_mask)
                        if b > 1:
                            previous_box_mask = np.zeros(seed_mask.shape) 
                            left_p, top_p, right_p, bottom_p = seed_class_list[tmp_seed_same].yolo_box
                            previous_box_mask[top_p:bottom_p+1, left_p:right_p+1]=1
                            box_overlap = tmp_box_mask * previous_box_mask
                            if np.sum(box_overlap) < 0.7 * np.sum(tmp_box_mask):
                                continue
                        box_candidate_index.append(box_index)
                        box_overlap_area.append(np.sum(tmp_box_mask * overall_mask))
                        box_coords.append([left, top, right, bottom])
                        box_conf.append(box.conf.tolist()[0])
                        box_label.append(box.cls.tolist()[0])
                if len(box_overlap_area) > 0:
                    box_conf_sort = sorted(range(len(box_conf)), key=lambda k: box_conf[k], reverse=True)
                    chosen_index = box_conf_sort[0]
                    seed_class_list[tmp_seed_same].update_yolo(box_coords[chosen_index], box_conf[chosen_index], box_label[chosen_index])
                    box_possess[box_candidate_index[chosen_index]] = 1
            elif len(overlap_num) == 0:
                continue
            else: # intersections
                tmp_seed_dict = {}
                tmp_seed_mask = np.zeros(seed_mask.shape)
                tmp_seedling_mask = np.zeros(seed_mask.shape)
                tmp_seedling_mask[tmp_coords[:, 0], tmp_coords[:, 1]] = 1
                overlap_num_seedmask = []
                overlap_num_flag = []
                for tmp_seed_index, tmp_seed_num in enumerate(overlap_num):
                    tmp_seed_dict[tmp_seed_num] = {}
                    tmp_region_index, tmp_seed_same = '', ''
                    for temp_region_index, temp_seed_same in trans_new.items():
                        if temp_seed_same == tmp_seed_num:
                            tmp_region_index = temp_region_index
                            tmp_seed_same = tmp_seed_num
                            tmp_seed_dict[tmp_seed_num] = tmp_seed_same
                            break

                    seed_class_list[tmp_seed_same].update_seed_coords(
                        seed_new_list[tmp_region_index].coords)
                    temp_single_seed_mask = np.zeros(seed_mask.shape)
                    temp_single_seed_mask[seed_new_list[tmp_region_index].coords[:, 0],
                    seed_new_list[tmp_region_index].coords[:, 1]] = 1
                    overlap_num_seedmask.append(temp_single_seed_mask)

                    box_candidate_index = []
                    box_overlap_area = []
                    box_coords = []
                    box_conf = []
                    box_label = []
                    for box_index, box in enumerate(tmp_yolo_box):
                        if box_possess[box_index] > 0:
                            continue
                        box_coord = box.xyxy.tolist()[0]
                        left, top, right, bottom = [int(m) for m in box_coord]
                        tmp_box_mask = np.zeros(seed_mask.shape)
                        tmp_box_mask[top:bottom+1, left:right+1]=1
                        previous_box = np.zeros(seed_mask.shape)
                        pre_left, pre_top, pre_right, pre_bottom = seed_class_list[tmp_seed_same].yolo_box
                        previous_box[pre_top:pre_bottom+1, pre_left:pre_right+1]=1
                        new_overlap_mask = tmp_box_mask * temp_single_seed_mask
                        overlap_pixel_num = np.sum(new_overlap_mask)
                        if overlap_pixel_num >= 0.85 * np.sum(temp_single_seed_mask):
                            a, b = ndi.label(tmp_box_mask * seed_mask)
                            if b > 1:
                                continue
                            box_candidate_index.append(box_index)
                            box_overlap_area.append(np.sum(tmp_box_mask * previous_box)/np.sum(tmp_box_mask))
                            box_coords.append([left, top, right, bottom])
                            box_conf.append(box.conf.tolist()[0])
                            box_label.append(box.cls.tolist()[0])
                    if len(box_overlap_area) > 0:
                        box_overlap_area_sort = sorted(range(len(box_overlap_area)), key=lambda k: box_overlap_area[k], reverse=True)
                        chosen_index = box_overlap_area_sort[0]
                        seed_class_list[tmp_seed_same].update_yolo(box_coords[chosen_index], box_conf[chosen_index], box_label[chosen_index])
                        box_possess[box_candidate_index[chosen_index]] = 1
                        overlap_num_flag.append(1)
                    else:
                        overlap_num_flag.append(0)

                for tmp_seed_index, tmp_seed_num in enumerate(overlap_num):
                    if overlap_num_flag[tmp_seed_index] > 0:
                        continue
                    temp_single_seed_mask = overlap_num_seedmask[tmp_seed_index]
                    box_candidate_index = []
                    box_overlap_area = []
                    box_coords = []
                    box_conf = []
                    box_label = []
                    tmp_seed_same = tmp_seed_dict[tmp_seed_num]
                    for box_index, box in enumerate(tmp_yolo_box):
                        if box_possess[box_index] > 0:
                            continue
                        box_coord = box.xyxy.tolist()[0]
                        left, top, right, bottom = [int(m) for m in box_coord]
                        tmp_box_mask = np.zeros(seed_mask.shape)
                        tmp_box_mask[top:bottom+1, left:right+1]=1
                        previous_box = np.zeros(seed_mask.shape)
                        pre_left, pre_top, pre_right, pre_bottom = seed_class_list[tmp_seed_same].yolo_box
                        previous_box[pre_top:pre_bottom+1, pre_left:pre_right+1]=1
                        new_overlap_mask = tmp_box_mask * temp_single_seed_mask
                        overlap_pixel_num = np.sum(new_overlap_mask)
                        if overlap_pixel_num >= 0.85 * np.sum(temp_single_seed_mask):
                            box_candidate_index.append(box_index)
                            box_overlap_area.append(np.sum(tmp_box_mask * previous_box)/np.sum(tmp_box_mask))
                            box_coords.append([left, top, right, bottom])
                            box_conf.append(box.conf.tolist()[0])
                            box_label.append(box.cls.tolist()[0])
                    if len(box_overlap_area) > 0:
                        box_overlap_area_sort = sorted(range(len(box_overlap_area)), key=lambda k: box_overlap_area[k], reverse=True)
                        chosen_index = box_overlap_area_sort[0]
                        seed_class_list[tmp_seed_same].update_yolo(box_coords[chosen_index], box_conf[chosen_index], box_label[chosen_index])
                        box_possess[box_candidate_index[chosen_index]] = 1

    seed_mask_pure = np.zeros(previous_mask.shape)
    for i in range(len(seed_class_list)):
        tmp_coords = seed_class_list[i].seed_coords
        seed_mask_pure[tmp_coords[:, 0], tmp_coords[:, 1]] = 1
    previous_mask = seed_mask_pure

    fig, ax = plt.subplots()
    plt.axis('off')
    ax.imshow(image_origion)
    for tmp_i in range(len(seed_class_list)):
        left, top, right, bottom = seed_class_list[tmp_i].yolo_box
        label_tmpbox = seed_class_list[tmp_i].yolo_label
        conf_tmpbox = seed_class_list[tmp_i].yolo_conf
        tmp_color = color_dict[label_tmpbox]
        width = right - left
        height = bottom - top
        ax.add_patch( 
            patches.Rectangle((left, top), width, height, edgecolor=tmp_color, fill=False)
        )
        plt.text(left+5, top-10, 
            round(conf_tmpbox,2), 
            color='white', 
            fontsize=5, 
            fontweight='bold', 
            ha='left', 
            bbox=dict(facecolor=tmp_color, edgecolor='none', pad=1.5))

    plt.savefig(os.path.join(out_dir, file_name), dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()  
    
    with open(os.path.join(yolo_adjust_dir, f'{file_prefix}.pkl'), 'wb') as f_pickle:
        pickle.dump(seed_class_list, f_pickle)
    

****************************************G7_20221026_1637.jpg****************************************
****************************************G7_20221026_2138.jpg****************************************
****************************************G7_20221027_0239.jpg****************************************
****************************************G7_20221027_0741.jpg****************************************
****************************************G7_20221027_1242.jpg****************************************
****************************************G7_20221027_1744.jpg****************************************
****************************************G7_20221027_2245.jpg****************************************
****************************************G7_20221028_0347.jpg****************************************
****************************************G7_20221028_0848.jpg****************************************
****************************************G7_20221028_1350.jpg*******************************

# output phase data

In [8]:
yolo_adjust_dir = './traits/roi_adjust/'

## seed level phase

In [9]:
phase_single = []
for tmp_pickle in os.listdir(yolo_adjust_dir):
    tmp_f = open(os.path.join(yolo_adjust_dir, tmp_pickle), 'rb')
    seed_class_list = pickle.load(tmp_f)
    tmp_single = []
    for i in range(len(seed_class_list)):
        tmp_single.append(seed_class_list[i].yolo_label)
    phase_single.append(tmp_single)

In [10]:
phase_single_df = pd.DataFrame(phase_single)

columns represent seed id (a total of 25 seeds in the germination experiment); <br>
rows represent each image id (5-hour interval); <br>
0 represents IMB (imbibition); <br>
1 represents PRO (protrusion); <br>
2 represents RE (radicle emergence); <br>
3 represents SE (seedling establishment); 

In [11]:
phase_single_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
8,2.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
9,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0


In [12]:
phase_single_df.to_csv(os.path.join(out_dir, 'phase_seed-level.csv'))

## seed lot level phase

In [13]:
timepoint_PRO = None
timepoint_RE = None
timepoint_SE = None

for i in range(len(phase_single_df)):
    row_data = phase_single_df.loc[i].values
    count_PRO = 0
    count_RE = 0
    count_SE = 0
    for tmp_single_phase in row_data:
        if tmp_single_phase >= 1:
            count_PRO += 1
        if tmp_single_phase >= 2:
            count_RE += 1
        if tmp_single_phase == 3:
            count_SE += 1
    proportion_PRO = count_PRO/len(row_data)
    proportion_RE = count_RE/len(row_data)
    proportion_SE = count_SE/len(row_data)
    
    if proportion_PRO >= 0.75 and timepoint_PRO is None:
        timepoint_PRO = i
    if proportion_RE >= 0.75 and timepoint_RE is None:
        timepoint_RE = i
    if proportion_SE >= 0.75 and timepoint_SE is None:
        timepoint_SE = i    

In [14]:
phase_seedlot_df = pd.DataFrame([timepoint_PRO, timepoint_RE, timepoint_SE])

In [15]:
phase_seedlot_df

,0
0,9
1,13
2,15


In [16]:
phase_seedlot_df.columns = ['index']
phase_seedlot_df.index = ['timepoint_PRO', 'timepoint_RE', 'timepoint_SE']

In [17]:
phase_seedlot_df

,index
timepoint_PRO,9
timepoint_RE,13
timepoint_SE,15


In [18]:
phase_seedlot_df.to_csv(os.path.join(out_dir, 'phase_seedlot-level.csv'))